In [1]:
# Now that we scraped our data and trained our model, lets use it to fill out a bracket!

In [2]:
import pickle
import pandas as pd 
import csv
import numpy as np 
import pickle

import sklearn

In [3]:
path_stats_basic19 = r"C:\Python\March Madness\NCAA_Team_Data_basic19.csv"
path_stats_basic18 = r"C:\Python\March Madness\NCAA_Team_Data_basic18.csv"

# df of all team stats for each year
df_stats19 = pd.read_csv(path_stats_basic19)
df_stats18 = pd.read_csv(path_stats_basic18)

# df picking out our stats we will use as features in our model, this is useful for our our match_stats functions below
df_features19 = df_stats19[['SCHOOL','SRS','FG%','3P%', 'FT%']]
df_features18 = df_stats18[['SCHOOL','SRS','FG%','3P%', 'FT%']]

In [4]:
# This csv file holds the first round of our tournament
path_tournament19 = r"C:\Python\March Madness\Bracket_Games19.csv"

In [5]:
# load our regression model using pickle
filename = 'finalized_model.sav'
model = pickle.load(open(filename,'rb'))

features = ['SRS','FG%','3P%','FT%']

In [6]:
# Calculates the match stats of each tournament game, similar to match_stats_reg_season()
def match_stats_tournament(lst,df_games,df_features):
    for index,row in df_games.iterrows():
        lst.append(df_features.loc[df_features['SCHOOL']==row[0]].values[0][1:].astype(float) - df_features.loc[df_features['SCHOOL']==row[1]].values[0][1:].astype(float))
        
    return lst

In [7]:
# Given the first round matchups, this function predicts the entire tournament saving it to a csv file

def predict_tournament(path_tournament,bracket_file,df_features): 
    
    # rounds list holds the indices for the first game of each tournament round in our csv file
    rounds = [0,33,50,59,64,67]


    # iterate through each round of the tournament so we can predict the winners of one round and update the bracket for the matches of the next round
    for r in rounds:

        # Two holder lists to keep track of our match stats and which teams advance respectively
        tournament_match_stats = [] 
        next_round = []

        # Fills the df_tournament data frame with the matches to be predicted for the given rounds
        df_tournament = pd.read_csv(path_tournament)

        # call our function to calculate the current rounds match stats
        match_stats_tournament(tournament_match_stats,df_tournament[r:],df_features)

        # Save the current rounds match stats to a df
        df_tournament_match_stats = pd.DataFrame(columns=features, data=tournament_match_stats)

        # predicts who will advance to the next round
        holdout_predicitons = model.predict(df_tournament_match_stats[features])


        # iterates through each matchup and picks out the predicted winner saving them to the next_round holder list
        count=0 
        for index,row in df_tournament[r:].iterrows():

            if holdout_predicitons[count] == 0:
                next_round.append(row[0])
                count+=1

            else:
                next_round.append(row[1])
                count+=1


        # Appends the winners of each round to our bracket csv file
        # First we will check if have a champion so we can properly crown them 
        if len(next_round)==1:
            with open(bracket_file,'a',newline='') as writefile:
                writer = csv.writer(writefile)
                writer.writerow(['CHAMPION:',next_round[0]])

        # Otherwise mark the next round for ease of reading and append the next matchups
        else:
            with open(bracket_file,'a',newline='') as writefile:
                writer = csv.writer(writefile)
                writer.writerow(['NEXT ROUND'])

                for i in range(len(next_round)-1):
                    if i%2==0:
                        writer.writerow([next_round[i],next_round[i+1]])
                    else:
                        pass

In [8]:
predict_tournament(path_tournament19,'Bracket_Games19.csv',df_features19)

In [9]:
# This function creates a new csv file of each matchup from the predicted bracket and the probability that Team A wins
def prob_predictor(df,path,csv_name):
    
    stat=[]
    probs=[]
    games=[]
    combine=[]
    
    df_tournament = pd.read_csv(path).dropna()
    
    match_stats_tournament(stat,df_tournament[0:-1],df)
    
    for index,row in df_tournament[0:63].iterrows():
        games.append([row[0],row[1]])
    
    for i in stat:
        probs.append(model.predict_proba(i.reshape(1,-1))[0])
    
    for i in range(len(games)):
        combine.append([games[i][0],games[i][1],probs[i][0]])
    
    with open(csv_name,'w',newline='') as writefile:
        writer = csv.writer(writefile)
        writer.writerow(['Team 1','Team 2','Probability'])
        
        for i in combine:
            writer.writerow(i)

In [11]:
prob_predictor(df_features19,path_tournament19,'Prediction_Probabilities.csv')